In [1]:
import nltk
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier, Lasso, RidgeClassifierCV
from sklearn.svm import LinearSVC
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from nltk.stem.porter import *
from sklearn.naive_bayes import BernoulliNB

/Users/rin/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Размеченные данные я брал из магазина Ситилинк в разделах "Сотовые телефоны" и "Смартфоны".
В папке лежит проект Scrapy.
Далее идет предобработка данных, т.к. в первом разделе я вытащил данные в виде массива, поэтому пришлось применять функцию ss


In [2]:
data = pd.read_json('feed.json')
data2 = pd.read_json('smart_citilink.json')

In [3]:
def ss(x):
    if len(x) == 0: return ''
    else: return x[0]

In [4]:
data.comment = data.comment.apply(ss)
data.positive = data.positive.apply(ss)
data.negative = data.negative.apply(ss)
data.rate = data.rate.apply(ss)

In [5]:
data = pd.concat([data,data2])

Если оценка выше 3ех - отзыв положительный

In [6]:
def rate(x):
    n = float(x)
    n = int(n)
    if n>2: return 1
    else: return 0
data.rate = data.rate.apply(rate)

In [22]:
data.head()

,comment,negative,positive,rate
0,"Советую к покупке,хорошая звонилка",Нет,"Дешёвый,простой в использовании",1
1,"купил как дополнительную трубку, отличная звон...",нет фонарика,"цена, блютуз, две сим, громкость и качество зв...",1
2,"Покупала папе, все устраивает. При умеренном и...","Интерфейс иногда соложноват, нужно привыкнуть","Все работает на отлично, не тормозит и не глючит",1
3,"Хорош смарт все работает без тормозов,качестве...",Нет,100%!!!,1
4,"Как телефон неплох, как смартфон ????",GPS не работает (за 30 минут на улице ни одног...,Телефон но не смартфон,0


In [8]:
data = data.dropna()

Если отзыв положительный, то беру лишь блок положительных качеств и суммирую с комментариями, если отзыв отрицательный, то соответственно суммирую с отрицательным блоком комментарии

In [9]:
feed = []
for i in range(data.shape[0]):
    if data.rate.iloc[i] == 1: feed.append((data.positive.iloc[i]+' '+data.comment.iloc[i]))
    else: feed.append(data.negative.iloc[i]+' '+data.comment.iloc[i])

In [10]:
y = data.rate.tolist()

Провожу токенизацию обучающей выборки:

In [16]:
import nltk
from nltk.tokenize import word_tokenize
X =  [word_tokenize(i) for i in feed]
X = [' '.join(i) for i in X]
X = [i.lower() for i in X]

Достаю тестовую выборку из файла и тоже провожу токенизацию. Токенизация проводится с помощью какого-то пакета из nltk, я не смог найти его конкретно, поэтому скачал сразу все пакеты. Если вы хотите что код ниже сработал и у вас есть свободное место на диске, то сперва выполните команду  "python -m nltk.downloader all"  в терминале


In [17]:
test=[]
import bs4
soup = bs4.BeautifulSoup(open("test.csv"))

for city in soup.find_all('review'):
    test.append(city.text)
    
test =  [word_tokenize(i) for i in test]
test = [' '.join(i) for i in test]
test = [i.lower() for i in test]

/Users/rin/anaconda/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /Users/rin/anaconda/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Проверяю почти стандартный пайплайн на кросс-валидации:

In [19]:
clf_pipeline = Pipeline(
            [("vectorizer", CountVectorizer(ngram_range=(3,5), analyzer='char_wb')),
             ("transformer", TfidfTransformer()),
            ("classifier", LinearSVC(class_weight='balanced'))]
        )
print cross_val_score(clf_pipeline, X, y).mean()

0.918027096921


Обучаю на тестовой выборке и создаю выходной файл с предсказаниями:

In [21]:
clf_pipeline.fit(X, y)
predicted = clf_pipeline.predict(test)
pp = ['pos' if i==1 else 'neg' for i in predicted]
with open('submit4.txt',"w") as fout:
    fout.write('Id,y\n')
    for i in np.arange(100):
        fout.write(str(i)+','+str(pp[i])+'\n')

Результат на Kaggle:

![title](scr.png)